In [9]:
import os, shutil
import numpy as np
import pandas as pd
import datetime as dt
from keras import models, layers, optimizers
from keras import preprocessing
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, Conv1D, MaxPooling1D
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

from utils import plot_val_acc_loss

## 1D convolution for sequence data

In [3]:
from keras.datasets import imdb
max_features = 10000
max_len = 500
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [4]:
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 3

In [6]:
model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 34s 205ms/step - loss: 0.7076 - acc: 0.5312 - val_loss: 0.6828 - val_acc: 0.5896
Epoch 2/10
157/157 [==============================] - 26s 166ms/step - loss: 0.6631 - acc: 0.6740 - val_loss: 0.6601 - val_acc: 0.6614
Epoch 3/10
157/157 [==============================] - 29s 183ms/step - loss: 0.6148 - acc: 0.7629 - val_loss: 0.5942 - val_acc: 0.7396
Epoch 4/10
157/157 [==============================] - 29s 182ms/step - loss: 0.5018 - acc: 0.8139 - val_loss: 0.4641 - val_acc: 0.8126
Epoch 5/10
157/157 [==============================] - 28s 180ms/step - loss: 0.3840 - acc: 0.8554 - val_loss: 0.4278 - val_acc: 0.8346
Epoch 6/10
157/157 [==============================] - 30s 188ms/step - loss: 0.3247 - acc: 0.8800 - val_loss: 0.4023 - val_acc: 0.8506
Epoch 7/10
157/157 [==============================] - 29s 187ms/step - loss: 0.2908 - acc: 0.8967 - val_loss: 0.4157 - val_acc: 0.8590
Epoch 8/10
157/157 [==============================] - 2

In [11]:
plot_val_acc_loss(history)

#### Convnets as preprocessing for RNNs

In [12]:
data_dir = r'C:\Users\Cyril\Downloads\jena_climate'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')
parser = lambda date: pd.datetime.strptime(date, '%d.%m.%Y %H:%M:%S')
df = pd.read_csv(fname, parse_dates=['Date Time'], date_parser=parser)
df.set_index('Date Time', inplace=True)

float_data = df.to_numpy()
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

C:\Users\Cyril\AppData\Local\Temp\ipykernel_4208\1530719735.py:3: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [13]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                            lookback // step,
                            data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [14]:
step = 3
lookback = 720
delay = 144
train_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=0,
                    max_index=200000,
                    shuffle=True,
                    step=step)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step)
test_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=300001,
                    max_index=None,
                    step=step)
val_steps = (300000 - 200001 - lookback) // 128
test_steps = (len(float_data) - 300001 - lookback) // 128

In [15]:
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
            input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, None, 32)          2272      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 32)         0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, None, 32)          5152      
                                                                 
 gru (GRU)                   (None, 32)                6336      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 13,793
Trainable params: 13,793
Non-trainable params: 0
__________________________________________________

In [17]:
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit(train_gen,
                    steps_per_epoch=500,
                    epochs=2)

Epoch 1/2
500/500 [==============================] - 81s 147ms/step - loss: 0.2970
Epoch 2/2
500/500 [==============================] - 72s 144ms/step - loss: 0.2844
